# atmodeller

## Tutorial 3: Monte Carlo experiment

We can devise a simple Monte Carlo approach to sample the probable atmospheres that can arise for different planetary conditions.

In [20]:
from atmodeller import InteriorAtmosphereSystem, Molecule, Planet, OCEAN_MOLES, SystemConstraint
from atmodeller.thermodynamics import PeridotiteH2O, NoSolubility, BasaltDixonCO2, MolarMasses
import numpy as np

In [21]:
molecules: list[Molecule] = []
molecules.append(Molecule(name='H2O', solubility=PeridotiteH2O(), solid_melt_distribution_coefficient=0))
molecules.append(Molecule(name='H2', solubility=NoSolubility(), solid_melt_distribution_coefficient=0))
molecules.append(Molecule(name='O2', solubility=NoSolubility(), solid_melt_distribution_coefficient=0))
molecules.append(Molecule(name='CO', solubility=NoSolubility(), solid_melt_distribution_coefficient=0))
molecules.append(Molecule(name='CO2', solubility=BasaltDixonCO2(), solid_melt_distribution_coefficient=0))
molecules

16:29:59 - atmodeller.core      - INFO      - Creating a new molecule: H2O
16:29:59 - atmodeller.core      - INFO      - Creating a new molecule: H2
16:29:59 - atmodeller.core      - INFO      - Creating a new molecule: O2
16:29:59 - atmodeller.core      - INFO      - Creating a new molecule: CO
16:29:59 - atmodeller.core      - INFO      - Creating a new molecule: CO2


[Molecule(name='H2O', solubility=<atmodeller.thermodynamics.PeridotiteH2O object at 0x11c783f10>, solid_melt_distribution_coefficient=0, elements={'H': 2, 'O': 1}, element_masses={'H': 0.0020158, 'O': 0.0159994}, formation_constants=(-3.0385132352941198, 13152.698529411768), molar_mass=0.018015200000000002),
 Molecule(name='H2', solubility=<atmodeller.thermodynamics.NoSolubility object at 0x11c7365d0>, solid_melt_distribution_coefficient=0, elements={'H': 2}, element_masses={'H': 0.0020158}, formation_constants=(0, 0), molar_mass=0.0020158),
 Molecule(name='O2', solubility=<atmodeller.thermodynamics.NoSolubility object at 0x11c312d10>, solid_melt_distribution_coefficient=0, elements={'O': 2}, element_masses={'O': 0.0319988}, formation_constants=(0, 0), molar_mass=0.0319988),
 Molecule(name='CO', solubility=<atmodeller.thermodynamics.NoSolubility object at 0x11c73ac50>, solid_melt_distribution_coefficient=0, elements={'C': 1, 'O': 1}, element_masses={'C': 0.0120107, 'O': 0.0159994}, for

In [22]:
planet: Planet = Planet()
interior_atmosphere: InteriorAtmosphereSystem = InteriorAtmosphereSystem(molecules=molecules, planet=planet)

16:29:59 - atmodeller.core      - INFO      - Creating a new planet
16:29:59 - atmodeller.core      - INFO      - Mantle mass (kg) = 4.208261222595111e+24
16:29:59 - atmodeller.core      - INFO      - Mantle melt fraction = 1.0
16:29:59 - atmodeller.core      - INFO      - Core mass fraction = 0.295334691460966
16:29:59 - atmodeller.core      - INFO      - Planetary radius (m) = 6371000.0
16:29:59 - atmodeller.core      - INFO      - Planetary mass (kg) = 5.972e+24
16:29:59 - atmodeller.core      - INFO      - Surface temperature (K) = 2000.000000
16:29:59 - atmodeller.core      - INFO      - Surface gravity (m/s^2) = 9.819973426224687
16:29:59 - atmodeller.core      - INFO      - Oxygen fugacity model (mantle) = IronWustiteBufferOneill
16:29:59 - atmodeller.core      - INFO      - Oxygen fugacity log10 shift = 0.000000
16:29:59 - atmodeller.core      - INFO      - Creating a new interior-atmosphere system
16:29:59 - atmodeller.core      - INFO      - Molecules = ['CO', 'H2', 'O2', 'CO

In [23]:
number_of_realisations: int = 2

# Parameters are normally distributed between bounds.
number_ocean_moles: np.ndarray = np.random.uniform(1, 10, number_of_realisations)
ch_ratios: np.ndarray = np.random.uniform(0.1, 1, number_of_realisations)
fo2_shifts: np.ndarray = np.random.uniform(-4, 4, number_of_realisations)

# Store the output in a list.
out: list[dict[str, float]] = []

for realisation in range(number_of_realisations):

    mass_H: float = number_ocean_moles[realisation] * OCEAN_MOLES * MolarMasses().H2
    mass_C: float = ch_ratios[realisation] * mass_H
    constraints: list[SystemConstraint] = [
        SystemConstraint(species="H", value=mass_H, field="mass"),
        SystemConstraint(species="C", value=mass_C, field="mass"),
    ]
    # Recall that changing attributes on the planet 'object' will be 'seen' by interior_atmosphere.
    planet.fo2_shift = fo2_shifts[realisation]
    interior_atmosphere.solve(constraints, fo2_constraint=True)
    out_realisation = interior_atmosphere.pressures_dict.copy()

    # Include the parameters in the output.
    out_realisation['number_ocean_moles'] = number_ocean_moles[realisation]
    out_realisation['ch_ratio'] = ch_ratios[realisation]
    out_realisation['fo2_shift'] = planet.fo2_shift

    out.append(out_realisation)

16:29:59 - atmodeller.core      - INFO      - Constraints: [SystemConstraint(species='H', value=3.723432469375954e+20, field='mass'),
 SystemConstraint(species='C', value=2.6092299668020645e+20, field='mass')]
16:29:59 - atmodeller.core      - INFO      - Mixed pressure and mass constraints so attempting to solve a non-linear system of equations
16:29:59 - atmodeller.core      - INFO      - Adding fO2 as an additional constraint using IronWustiteBufferOneill with fO2_shift = 1.99
16:29:59 - atmodeller.core      - INFO      - 2 additional (not pressure) constraint(s) are necessary to solve the system
16:29:59 - atmodeller.core      - INFO      - Row 00: Reaction 0: 1.0 CO + 0.5 O2 = 1.0 CO2
16:29:59 - atmodeller.core      - INFO      - Row 01: Reaction 1: 1.0 H2 + 0.5 O2 = 1.0 H2O
16:29:59 - atmodeller.core      - INFO      - Row 02: Setting O2 partial pressure
16:29:59 - atmodeller.core      - INFO      - Adding constraint from mass balance: H
16:29:59 - atmodeller.core      - INFO    

The simulation data is output as:

In [24]:
out

[{'CO': 52.44161304176824,
  'H2': 0.3239052107650134,
  'O2': 3.820856184470401e-06,
  'CO2': 78.79186625815697,
  'H2O': 2.184410773322142,
  'number_ocean_moles': 2.4023098318015412,
  'ch_ratio': 0.7007593096590713,
  'fo2_shift': 1.989984533453283},
 {'CO': 335.1656506083731,
  'H2': 92.25448099074877,
  'O2': 1.0320468183986398e-08,
  'CO2': 26.17183678884906,
  'H2O': 32.33502484687907,
  'number_ocean_moles': 9.586832588910118,
  'ch_ratio': 0.6611067020598059,
  'fo2_shift': -0.5784767585319255}]